In [ ]:
# coding=utf-8
from gensim import corpora, models
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import pandas as pd


In [ ]:
df = pd.read_csv('../input/after_preprocess_news.csv')
news = df.content
news = news.dropna()
train = [x.split() for x in news]
train

In [ ]:
# 构造词典
dictionary = corpora.Dictionary(train)
feature_cnt = len(dictionary.token2id)  # 词典中词的数量
dictionary.save('dict.txt')  # 保存生成的词典,用于以后加载
# dictionary=Dictionary.load('dict.txt')#加载词典

# 基于词典，将【分词列表集】转换成【向量集】，形成【语料库】
corpus = [dictionary.doc2bow(text) for text in train]

# 使用【TF-IDF模型】处理语料库
tfidf_model = models.TfidfModel(corpus)

# 打印模型参数：文档数量与语料库单词数
print(tfidf_model)

# 存储通过tf-idf转化过的文档
with open('tfidf_doc.txt', 'w', encoding='utf-8') as fr:
    for doc in tfidf_model[corpus]:
        fr.write(doc.__str__() + '\n')

In [4]:
import gensim
import matplotlib.pyplot as plt
from gensim.models import CoherenceModel

coherence = []
# 假设主题数量在1-20中进行选择
for n in range(1, 21):   
    # Multi-core implementation:
    lda = gensim.models.LdaMulticore(corpus, num_topics = n, id2word = dictionary, passes = 10, workers = 2)
    
    # ompute coherence for each lda model with different number of topics
    cohm = CoherenceModel(model = lda, corpus = corpus, dictionary = dictionary, coherence = 'u_mass')
    coh = cohm.get_coherence()
    coherence.append(coh)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\song\AppData\Local\Programs\Python\Python38\Lib\site-packages\IPython\core\interactiveshell.py", line 3444, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\song\AppData\Local\Temp/ipykernel_9424/138430355.py", line 9, in <module>
    lda = gensim.models.LdaMulticore(corpus, num_topics = n, id2word = dictionary, passes = 10, workers = 2)
  File "C:\Users\song\AppData\Local\Programs\Python\Python38\Lib\site-packages\gensim\models\ldamulticore.py", line 186, in __init__
    super(LdaMulticore, self).__init__(
  File "C:\Users\song\AppData\Local\Programs\Python\Python38\Lib\site-packages\gensim\models\ldamodel.py", line 520, in __init__
    self.update(corpus, chunks_as_numpy=use_numpy)
  File "C:\Users\song\AppData\Local\Programs\Python\Python38\Lib\site-packages\gensim\models\ldamulticore.py", line 316, in update
    process_result_queue(force=True)
  File "C:\Users\song\AppData\Local\Programs\Python\P

TypeError: object of type 'NoneType' has no len()

In [ ]:
print(coherence)
topic = [i for i in range(1,21)]
plt.plot(topic,coherence)
plt.show()

In [ ]:
# 利用lda模型对语料库进行建模，设置模型的主题的个数、迭代次数
lda = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=5, chunksize=1000, iterations=2000)
# 打印LDA模型参数
print(lda)

# 存储LDA文档结果：存储文档-主题分布
with open('doc_dis.txt', 'w') as f1:
    # for doc in lda_corpus:
    for doc in lda[corpus]:
        f1.write(doc.__str__() + '\n')
        f1.flush()
        
# 存储LDA文档结果：存储主题-词分布
with open('topic_dis.txt', 'w') as f2:
    for topic in lda.get_topics():
        f2.write(topic.__str__() + '\n')
        f2.flush()

In [ ]:
for topic in lda.print_topics(num_topics=5, num_words=30):
    print(topic)

In [ ]:
# Visualize the topics
import pyLDAvis.gensim_models
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

vis = pyLDAvis.gensim_models.prepare(lda, corpus, dictionary,mds='mmds')
pyLDAvis.save_html(vis, './vis.html')
